In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
## Environment
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/models'):
    main_path = p[:-len('/models')]
sys.path[0] = main_path

import os, gc
from termcolor import colored
import pandas as pd
import numpy as np
import joblib
from src import config, training

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
# DATA PREPARATION
# ===== LOAD DATA ======
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
print(f'Metadata: {metadata.shape}')

train_labels = pd.read_csv(config.DATA_DIR + 'train_labels.csv')
print(f'Train labels: {train_labels.shape}')

valid_labels = pd.read_csv(config.DATA_DIR + 'val_labels.csv')
print(f'Train labels: {valid_labels.shape}')

# Combine train and valid labels
trvl_labels = pd.concat([train_labels, valid_labels], axis = 0)

submission = pd.read_csv(config.DATA_DIR + 'submission_format.csv')
print(f'Submission: {submission.shape}')

# ===== FILE PATHS OF SAMPLES =====
train_files = metadata[metadata.split == 'train']['features_path'].to_dict()
valid_files = metadata[metadata.split == 'val']['features_path'].to_dict()
test_files = metadata[metadata.split == 'test']['features_path'].to_dict()
# Train & Valid files
trva_files = train_files.copy()
trva_files.update(valid_files)
# All files
all_test_files = valid_files.copy()
all_test_files.update(test_files)

# Define SAM testbed files
sam_files = metadata[(metadata.instrument_type == 'sam_testbed') & (metadata.split == 'train')]['features_path']
sam_files = sam_files.to_dict()

# Get the names of the target columns in a list
target_labels_list = [i for i in train_labels.columns if i not in ['sample_id']]
print(target_labels_list)

# SAM testbed labels
sam_labels = train_labels.drop(train_labels.tail(len(sam_files)).index)
sam_labels = pd.concat([sam_labels, valid_labels], axis=0)
print(f'Labels w/o SAM : {sam_labels.shape}')

Metadata: (1570, 5)
Train labels: (766, 11)
Train labels: (293, 11)
Submission: (804, 11)
['basalt', 'carbonate', 'chloride', 'iron_oxide', 'oxalate', 'oxychlorine', 'phyllosilicate', 'silicate', 'sulfate', 'sulfide']
Labels w/o SAM : (1047, 11)


In [4]:
SUB_NAME = 'ens_0097386'

In [5]:
def ens_model(label):
    if label == 'oxalate':
        MODEL_CLF = 'fts_mra_tempmz_LR_reg_trvl_' + label + '.joblib.dat'
        FTS_NAME = 'fts_mra_tempmz'
        VT_SAMPLE = FTS_NAME + '_vlte.csv'
    elif label == 'carbonate':
        MODEL_CLF = 'fts_mra_tempmz_XGB_trvl_' + label + '.joblib.dat'
        FTS_NAME = 'fts_mra_tempmz'
        VT_SAMPLE = FTS_NAME + '_vlte.csv'
    elif label == 'silicate':
        MODEL_CLF = 'fts_mra_tempmz_XGB_opt_trvl_' + label + '.joblib.dat'
        FTS_NAME = 'fts_mra_tempmz'
        VT_SAMPLE = FTS_NAME + '_vlte.csv'
    elif label in ['sulfide', 'iron_oxide']:
        MODEL_CLF = 'fts_mra_tempmz_cntpk_mratt_XGB_opt_trvl_' + label + '.joblib.dat'
        FTS_NAME = 'fts_mra_tempmz_cntpk_mratt'
        VT_SAMPLE = FTS_NAME + '_vlte.csv'
    else:
        MODEL_CLF = 'fts_mra_tempmz_cntpk_mratt_slope_XGB_opt_trvl_' + label + '.joblib.dat'
        FTS_NAME = 'fts_mra_tempmz_cntpk_mratt_slope'
        VT_SAMPLE = FTS_NAME + '_vlte.csv'

    return MODEL_CLF, FTS_NAME, VT_SAMPLE

In [6]:
for label in target_labels_list:
    MODEL_CLF, FTS_NAME, VT_SAMPLE = ens_model(label)

    # Load saved model
    clf = joblib.load(os.path.join(config.MODELS_DIR, MODEL_CLF))
    
    # Load valid & test data set
    X_vlte = pd.read_csv(os.path.join(config.DATA_DIR_OUT,VT_SAMPLE))
    print(f'Valid & Test: {X_vlte.shape}')
    
    submission[label] = clf.predict_proba(X_vlte)[:,1]

# Save submission file
submission.to_csv(os.path.join(config.MODELS_DIR, SUB_NAME))

Valid & Test: (804, 1981)


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Valid & Test: (804, 1584)


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Valid & Test: (804, 1981)


...

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Valid & Test: (804, 1981)


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Valid & Test: (804, 1980)


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [7]:
clf_loss, clf_loss_avg = training.compute_valid_loss(submission.iloc[:len(valid_files)],
                       valid_files,
                       valid_labels, 
                       target_labels_list,
                       SUB_NAME)
print(clf_loss_avg)

(293, 11)
0.009738676620661657
